In [1]:
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
import urllib.request


In [3]:
import datetime

In [4]:
import requests # Импорт библиотеки для запросов к серверу
from bs4 import BeautifulSoup # Импорт библиотеки для автоматического парсинга странички

In [5]:
df=pd.read_csv('url_schedule_v2 - schedule.csv', header=None)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       64 non-null     object
dtypes: object(1)
memory usage: 640.0+ bytes


In [7]:
url_list=list(df[0])

In [8]:
df=pd.DataFrame()
for i in url_list:
    req = requests.get(i, verify=False) # GET-запрос
    soup = BeautifulSoup(req.text, 'lxml')
    heading_2=soup.find('h2')
    df.loc[i, 'uc'] =i.split('=')[1]
    df.loc[i, 'date'] = (datetime.datetime.now()).strftime('%d/%m/%y')
    if "обновляем"  in (heading_2.text):
        df.loc[i, 'schedule'] = 0
    else:
        df.loc[i, 'schedule'] = 1
        cnt=0
        for paragraph in soup.find_all('div', attrs={'class': 'dscell dswuc'}):
            cnt = cnt +1 
            df.loc[i, 'group_cnt'] = int(cnt) #print(heading_2) 

/home/elena314/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'languagelink.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/elena314/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.languagelink.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/elena314/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'languagelink.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/elena314/anaconda3/li

In [9]:
dt_now = datetime.datetime.now()

In [10]:
now = str(dt_now)

In [11]:
file_name='schedule' + now + '.csv'

In [12]:
df.to_csv(file_name, decimal='.')

In [13]:
df=df.sort_values(by='group_cnt', ascending = False)

# вывод итоговой таблицы с числом групп в расписании на дату

In [14]:
df =  df.reset_index()
df.loc[:,['uc', 'date', 'group_cnt']].to_csv('schedule_v1709.csv',decimal=',')

In [15]:
df['group_cnt'] = df['group_cnt'].fillna(0)

In [16]:
df=df.reset_index()

In [17]:
df.rename(columns={'index': 'url'}, inplace=True)

In [18]:
df

,level_0,url,uc,date,schedule,group_cnt
0,0,https://languagelink.ru/schedule/?center=aleks...,alekseevskaya,23/10/22,1.0,50.0
1,1,https://languagelink.ru/schedule/?center=dmitr...,dmitrovskaya-timiryazevskaya,23/10/22,1.0,50.0
2,2,https://languagelink.ru/schedule/?center=sokol,sokol,23/10/22,1.0,50.0
3,3,https://languagelink.ru/schedule/?center=skhodnya,skhodnya,23/10/22,1.0,50.0
4,4,https://languagelink.ru/schedule/?center=izmay...,izmaylovo,23/10/22,1.0,50.0
...,...,...,...,...,...,...
59,59,https://languagelink.ru/schedule/?center=teply...,teply_stan-konkovo,23/10/22,0.0,0.0
60,60,https://languagelink.ru/schedule/?center=ufa,ufa,23/10/22,0.0,0.0
61,61,https://languagelink.ru/schedule/?center=uralsk,uralsk,23/10/22,0.0,0.0
62,62,https://www.languagelink.ru/schedule/?center=a...,aktau,23/10/22,0.0,0.0


In [19]:
df['schedule']=df['schedule'].astype('int32')

In [20]:
print('всего расписание прислали школ', df['schedule'].sum())

всего расписание прислали школ 36


In [21]:
hits=pd.read_csv('LanguageLink.ru-Популярное (3) - Отчет.csv')

In [22]:
hits=hits.rename(columns={'Адрес страницы': 'url'})

In [23]:
hits

,url,Просмотры,Посетители
0,https://languagelink.ru/schedule/?center=onlin...,374,297
1,https://languagelink.ru/schedule/?center=143#ucf,332,262
2,https://languagelink.ru/schedule/?center=novos...,181,141
3,https://languagelink.ru/schedule/?center=saint...,178,145
4,https://languagelink.ru/schedule/?center=onlin...,167,95
...,...,...,...
923,https://languagelink.ru/schedule/?center=zelen...,1,1
924,https://languagelink.ru/schedule/?center=zelen...,1,1
925,https://languagelink.ru/schedule/?center=zelen...,1,1
926,https://languagelink.ru/schedule/?center=zelen...,1,1


In [24]:
hits['url']=hits['url'].apply(lambda x: x.split('#')[0])

In [27]:
all=hits.merge(df, on='url', how='right')

In [29]:
all['predict_lead']=all['Посетители']*0.16 # 16% конверсия в заявку, для простоты считаем ее одинаковой

In [30]:
all.head(50)

,url,Просмотры,Посетители,level_0,uc,date,schedule,group_cnt,predict_lead
0,https://languagelink.ru/schedule/?center=aleks...,118.0,89.0,0,alekseevskaya,23/10/22,1,50.0,14.24
1,https://languagelink.ru/schedule/?center=dmitr...,115.0,91.0,1,dmitrovskaya-timiryazevskaya,23/10/22,1,50.0,14.56
2,https://languagelink.ru/schedule/?center=sokol,78.0,67.0,2,sokol,23/10/22,1,50.0,10.72
3,https://languagelink.ru/schedule/?center=skhodnya,42.0,29.0,3,skhodnya,23/10/22,1,50.0,4.64
4,https://languagelink.ru/schedule/?center=izmay...,49.0,43.0,4,izmaylovo,23/10/22,1,50.0,6.88
5,https://languagelink.ru/schedule/?center=sviblovo,123.0,85.0,5,sviblovo,23/10/22,1,42.0,13.60
6,https://languagelink.ru/schedule/?center=zelen...,153.0,112.0,6,zelenograd-16_mkrn,23/10/22,1,40.0,17.92
7,https://languagelink.ru/schedule/?center=khodynka,113.0,84.0,7,khodynka,23/10/22,1,39.0,13.44
8,https://languagelink.ru/schedule/?center=mititchi,88.0,65.0,8,mititchi,23/10/22,1,37.0,10.40
9,https://languagelink.ru/schedule/?center=khimki,59.0,48.0,9,khimki,23/10/22,1,35.0,7.68


In [31]:
all.to_csv('predict_leads_on_schedule_and_visitors.csv', decimal=',')